Фаррахов Фанур 05-204 Вариант 16.

In [8]:
import math
import numpy as np
import sympy as sp
import IPython.display as dp

Напишем для начала методы решений СЛАУ...

In [125]:
def simple_iteration_solver(B, b, x0=None, max_iter=1000):
    B = np.array(B, dtype=float)
    b = np.array(b, dtype=float)
    n = B.shape[0]

    x_current = np.zeros(n) if x0 is None else np.array(x0, dtype=float)

    for _ in range(max_iter):
        x_new = np.dot(B, x_current) + b
        x_current = x_new.copy()

    return x_current  

In [93]:
def seidel(A, b, x0=None, epsilon=1e-6, max_iter=1000):
    A = np.array(A, dtype=float)
    b = np.array(b, dtype=float)
    n = A.shape[0]

    x_current = np.zeros(n) if x0 is None else np.array(x0, dtype=float)

    Al = np.tril(A)
    Au = np.triu(A, k=1)
    B = -1 * np.dot(np.linalg.inv(Al), Au)
    b= np.dot(np.linalg.inv(Al), b)
    for _ in range(max_iter):
        x_new = np.dot(B, x_current) + b

        if np.linalg.norm(x_new - x_current, np.inf) < epsilon * 1e-1:
            x_current = x_new
            break
        x_current = x_new

    return x_current

In [159]:
def gradient_descent(A, b, x0=None, max_iter=1000):
    A = np.array(A, dtype=float)
    b = np.array(b, dtype=float)
    n = A.shape[0]

    x_current = np.zeros(n) if x0 is None else np.array(x0, dtype=float)

    for k in range(max_iter):
        r = b - np.dot(A, x_current)
        tau = np.dot(r, r) / np.dot(r, (np.dot(A, r)))
        
        x_new = x_current + tau * r
        x_current = x_new
        
    return x_current

In [43]:
def gauss_partial_pivot(A, b):
    A = np.array(A, dtype=float)
    b = np.array(b, dtype=float)
    n = len(b)
    aug = np.hstack([A, b.reshape(-1, 1)])

    for i in range(n):
        pivot = np.argmax(np.abs(aug[i:, i])) + i
        aug[[i, pivot]] = aug[[pivot, i]]  # Замена строчек местами

        for j in range(i + 1, n):
            aug[j] = aug[j] - aug[i] * (aug[j, i] / aug[i, i])

    x = np.zeros(n)
    for i in range(n-1, -1, -1):
        x[i] = (aug[i, -1] - np.dot(aug[i, i+1:n], x[i+1:])) / aug[i, i]

    return x

In [137]:
def get_N_withAccuracy(B, b, epsilon):
    row_sums = np.linalg.norm(B, np.inf)
    q = np.max(row_sums)
    if q >= 1:
        raise ValueError("Метод не сходится: норма матрицы >= 1")
    
    # Норма разности первого приближения (x1 = b) и начального (x0 = (0, 0, ...))
    # delta = || x1 - x0 ||
    delta = np.linalg.norm(b, np.inf)
    
    n = np.ceil(np.log(epsilon * (1-q) / delta) / np.log(q)).astype(int)
    return max(n, 1)

Оценка погрешности для градиентного спуска

$||x^* - x^k||_2 \lt \frac{||r^0||_2}{m}\left(\frac{M-m}{M+m}\right)^k$, где $r^0 = b - Ax^0$, $M$ и $m$ - макс. и мин. собственные значения м-цы A

In [156]:
def get_N_for_gradient(A, b, epsilon, x0=None):
    A = np.array(A, dtype=float)
    b = np.array(b, dtype=float)
    x0 = np.zeros(len(b)) if x0 is None else np.array(x0, dtype=float)
    
    r0 = b - np.dot(A, x0)
    r0_norm = np.linalg.norm(r0, 2)
    eigvals = np.linalg.eigvalsh(A)
    m = np.min(eigvals)
    M = np.max(eigvals)

    n = np.ceil(np.log(epsilon * m / r0_norm) / np.log((M-m)/(M+m))).astype(int)
    return max(n,1)

**Задача 1.**

Решить с точностью $10^{-4}$ систему $𝐴𝑥 = 𝑦$ 4-ого порядка методами простой итерации, Зейделя 2 вариант, метод градиентного спуска. Для каждого из указанных методов найти невязку приближенного решения и оценить погрешность. Сравнить полученное приближенное решение с решением, найденным методом Гаусса с частичным выбором ведущего элемента.

$$
\begin{cases}
0.11 x_1 + 1.13 x_2 - 0.17 x_3 + 0.18 x_4 = 1.00 \\
0.13 x_1 - 1.17 x_2 + 0.18 x_3 + 0.14 x_4 = 0.13 \\
0.11 x_1 - 1.05 x_2 - 0.17 x_3 - 0.15 x_4 = 0.11 \\
0.15 x_1 - 0.05 x_2 + 0.18 x_3 - 0.11 x_4 = 1.00 \\
\end{cases}
$$

In [11]:
A1 = np.array([[0.11, 1.13, -0.17, 0.18], \
               [0.13, -1.17, 0.18, 0.14], \
               [0.11, -1.05, -0.17, -0.15], \
               [0.15, -0.05, 0.18, -0.11]])
y1 = [1, 0.13, 0.11, 1]

**Метод простой итерации**

Покажем, что метод простой итерации не сходиться для этой системы

In [70]:
from scipy.optimize import minimize

def non_converging_method(A, method = 'Powell', norm_ord=np.inf):
    # B = E - tau * C * A
    def objective(x):
        tau = x[0]
        C = x[1:].reshape((4, 4))
        I = np.eye(4)
        B = I - tau * C @ A
        # Бесконечная норма: максимум суммы абсолютных значений по строкам
        return np.linalg.norm(B, ord=norm_ord)
    
    # A = M - N
    # B = tau * M^-1 * N
    def objective2(x):
        tau = x[0]
        M = x[1:].reshape((4, 4))
        N = M - A
        B = np.dot(np.linalg.inv(M), N)
        return np.linalg.norm(B, ord=norm_ord)
    
    # Начальное приближение: tau=1, C=единичная матрица
    x0 = np.zeros(17)
    x0[0] = 1.0
    x0[1:] = np.eye(4).flatten()
    
    # Оптимизация с ограничением Norm[B, Inf] < 1 (если нужно)
    constraints = [{'type': 'ineq', 'fun': lambda x: 1 - objective(x)}]
    method = 'Powell'
    
    # result = minimize(objective, x0, method=method, constraints=constraints)
    result = minimize(objective, x0, method=method)
    
    print("Результат для первого способа:")
    print("Минимальная норма:", result.fun)
    print("tau =", result.x[0])
    print("Матрица F:")
    print(result.x[1:].reshape((4, 4)))
    
    result = minimize(objective2, x0, method=method)
    
    print("\nРезультат для второго способа:")
    print("Минимальная норма:", result.fun)
    print("tau =", result.x[0])
    print("Матрица F:")
    print(result.x[1:].reshape((4, 4)))
non_converging_method(A1)

Результат для первого способа:
Минимальная норма: 1.0000000183302036
tau = 4.036191924978419e-08
Матрица F:
[[-0.35895119 -1.01109386  0.65288136  1.02856022]
 [ 2.58792896  0.65792618  0.66520013  0.62867712]
 [ 0.71024892 -0.08680946  0.86109362 -0.12143385]
 [-0.08230933 -0.1464417   0.02962904 -1.1816352 ]]

Результат для второго способа:
Минимальная норма: 1.0000000000154632
tau = 1.0
Матрица F:
[[ 3.87551507e+11 -4.22632285e+13  1.66125193e+13  1.98174916e+05]
 [ 6.83849155e+13  1.38578566e+00 -1.15198244e-03  3.78613626e-02]
 [-1.14274325e+14  6.76751462e+13 -1.08669186e+11  1.19149268e+09]
 [-1.46987312e+11 -3.52237157e+07  3.61267180e+05  3.18710971e+10]]


Отсюда видно, что минимальная норма $\ge 1$, поэтому метод не сходится для этой матрицы

**Метод Зейделя 2-ой вариант**

$Ax=y \quad \Rightarrow \quad A^T Ax = A^T y \quad (A^T A = A_l + A_u) \quad \Rightarrow \quad A_l x^{k+1} + A_u x^k = A^T y \quad \Rightarrow \quad x^{k+1} = -A_l^{-1} A_u x^k + A^T y$

$S = A^T A$ - симметричная. Для сходимости метода нужно, чтобы матрица $S$ была положительно определенна

In [37]:
S = np.dot(A1.T, A1)
for i in range(1, S.shape[0] + 1):
    print(f"{i}-ый минор м-цы S = {np.linalg.det(S[:i, :i])}")
S

1-ый минор м-цы S = 0.06360000000000002
2-ый минор м-цы S = 0.21581024
3-ый минор м-цы S = 0.023280070320000006
4-ый минор м-цы S = 0.0016599398983695994


array([[ 6.3600e-02, -1.5080e-01,  1.3000e-02,  5.0000e-03],
       [-1.5080e-01,  3.7508e+00, -2.3320e-01,  2.0260e-01],
       [ 1.3000e-02, -2.3320e-01,  1.2260e-01,  3.0000e-04],
       [ 5.0000e-03,  2.0260e-01,  3.0000e-04,  8.6600e-02]])

In [126]:
seidel1 = seidel(S, np.dot(A1.T, y1), epsilon=1e-3, max_iter=500)

**Метод градиентного спуска**

$Ax=b$

$x^0 = (x^0_1, \ldots, x^0_n) \in \mathbb{R}^n$

$x^{k+1} = x^k + \tau_k r^k$, где $r^k = b - A x^k$


$\tau_k = \frac{\left(r^k, r^k \right)}{(r^k, A r^k)}$

Чтобы метод сходился, матрица A должна быть симметричной и положительно определенной

Чтобы привести м-цу А к симметричной нужно домножить слева на транспонированную. Получим такую систему $Sx = A^T b$, где $S = A^T A$

In [157]:
max_iter = get_N_for_gradient(S, np.dot(A1.T, y1), 1e-3)
grad1 = gradient_descent(S, np.dot(A1.T, y1),max_iter=max_iter)

In [158]:
gauss1 = gauss_partial_pivot(A1, y1)

In [164]:
print("Метод Зейделя:")
print(f"Решение: {seidel1}")
print(f"Невязка: {np.dot(A1, seidel1) - y1}, норма: {np.linalg.norm(np.dot(A1, seidel1) - y1, np.inf):.2e}")

print("\nМетод градиентного спуска:")
print(f"Решение: {grad1}")
print(f"Невязка: {np.dot(A1, grad1) - y1}, норма: {np.linalg.norm(np.dot(A1, grad1) - y1, np.inf):.2e}")

print("\nМетод Гаусса (с частичным выбором):")
print(f"Решение: {gauss1}")
print(f"Невязка: {np.dot(A1, gauss1) - y1}, норма: {np.linalg.norm(np.dot(A1, gauss1) - y1, np.inf):.2e}\n")

# Оценка погрешности
error_seidel = np.linalg.norm(seidel1 - gauss1)
error_gd = np.linalg.norm(grad1 - gauss1)
print(f"Погрешность (Зейдель vs Гаусс): {error_seidel:.2e}")
print(f"Погрешность (Градиент vs Гаусс): {error_gd:.2e}")

Метод Зейделя:
Решение: [ 5.72169017  0.51589055  0.49622359 -0.71104785]
Невязка: [-4.38173762e-06  1.32612787e-06  7.38811921e-09 -5.49239171e-06], норма: 5.49e-06

Метод градиентного спуска:
Решение: [ 5.72171614  0.51589402  0.49622751 -0.71105747]
Невязка: [-2.03792849e-09  4.64932537e-10 -2.94143389e-09 -5.88817806e-09], норма: 5.89e-09

Метод Гаусса (с частичным выбором):
Решение: [ 5.72171617  0.51589402  0.49622751 -0.71105748]
Невязка: [ 0.00000000e+00 -2.22044605e-16  4.16333634e-17  0.00000000e+00], норма: 2.22e-16

Погрешность (Зейдель vs Гаусс): 2.82e-05
Погрешность (Градиент vs Гаусс): 3.08e-08


**Задача 2.**

Решить систему линейных уравнений методом квадратных корней с точностью до $0.001$

$$
\begin{cases}
\begin{aligned}
1.36x_1 + 0.92x_2 - 1.87x_3 &= 2.15, \\
0.92x_1 - 2.24x_2 + 0.77x_3 &= -2.06, \\
-1.87x_1 + 0.77x_2 - 1.16x_3 &= 0.17
\end{aligned}
\end{cases}
$$

In [98]:
A2 = np.array([
    [1.36, 0.92, -1.87],
    [0.92, -2.24, 0.77],
    [-1.87, 0.77, -1.16]])
b2 = np.array([2.15, -2.06, 0.17])

In [165]:
def sq_root(A, b):
    A = np.array(A, dtype=float)
    b = np.array(b, dtype=float)
    n = len(b)

    def sq_decomposition(A):
        S = np.zeros((n, n))
        d = np.zeros(n)

        for i in range(n):
            diagonal = A[i, i] - np.sum(S[:i, i]**2 * d[:i])
            d[i] = np.sign(diagonal)
            S[i, i] = np.sqrt(d[i] * diagonal)

            for j in range(i+1, n):
                S[i,j] = (A[i,j] - np.sum(S[:i,i] * S[:i,j] * d[:i])) / (S[i,i] * d[i])
        return S, d

    def solve_forward(S, d, b):
        y = np.zeros(n)
        M = np.dot(np.array(S).T, np.diag(d))
        for i in range(n):
            y[i] = (b[i] - np.dot(M[i,:i], y[:i])) / M[i,i]
        return y

    def solve_backward(S, d, y):
        x = np.zeros(n)
        M = np.array(S)
        for i in range(n-1, -1, -1):
            x[i] =  (y[i] - np.dot(M[i, i+1:], x[i+1:])) / M[i,i]
        return x

    S, d = sq_decomposition(A)
    y = solve_forward(S, d, b)
    x = solve_backward(S, d, y)
    return x

In [122]:
sq_root(A2, b2)

array([ 0.50466066,  1.0324672 , -0.27475491])

In [109]:
np.linalg.solve(A2, b2)

array([ 0.50466066,  1.0324672 , -0.27475491])

**Задача 3.**

Методом итерации решить систему линейных уравнений с точностью до 0.001, предварительно оценив число необходимых для этого шагов.

$$
\begin{cases}
x_1 = 0.14x_1 + 0.23x_2 + 0.18x_3 + 0.17x_4 - 1.42, \\
x_2 = 0.12x_1 - 0.14x_2 + 0.08x_3 + 0.09x_4 - 0.83, \\
x_3 = 0.16x_1 + 0.24x_2 - 0.35x_4 + 1.21, \\
x_4 = 0.23x_1 - 0.08x_2 + 0.05x_3 + 0.25x_4 + 0.65.
\end{cases}
$$

In [147]:
B3 = np.array([
    [0.14, 0.23, 0.18, 0.17],
    [0.12, -0.14, 0.08, 0.09],
    [0.16, 0.24, 0.00, -0.35],
    [0.23, -0.08, 0.05, 0.25]])
b3 = np.array([-1.42, -0.83, 1.21, 0.65])
epsilon = 0.001
max_iter = get_N_withAccuracy(B3, b, epsilon)

In [146]:
simple_iteration_solver(B3, b3, max_iter=max_iter)

array([-1.65444305, -0.82334072,  0.57771287,  0.48564133])

**Задача 4.**

Методом зейделя решить с точностью $0.001$ ситемы линейных уравнений, приведя ее к виду удобному для итерации.


$$
\begin{cases}
\begin{aligned}
3.8x_1 + 4.1x_2 - 2.3x_3 &= 4.8, \\
-2.1x_1 + 3.9x_2 - 5.8x_3 &= 3.3, \\
1.8x_1 + 1.1x_2 - 2.1x_3 &= 5.8.
\end{aligned}
\end{cases}
$$

In [63]:
A4 = np.array([[3.8, 4.1, -2.3], \
               [-2.1, 3.9, -5.8], \
               [1.8, 1.1, -2.1]])
y4 = [4.8, 3.3, 5.8]

In [92]:
seidel(np.dot(A4.T, A4), np.dot(A4.T, y4), epsilon=1e-3, max_iter=500)

array([ 1.60333082, -1.53890157, -2.18485292])

In [65]:
gauss_partial_pivot(A4, y4)

array([ 1.60940739, -1.55265204, -2.19570663])